In [1]:
from data_helper import *
from Dataset import *
import matplotlib.pyplot as plt
from base_mode import *
from build_vocab import Vocabulary
#from gensim.models import Word2Vec
from torch.nn.utils.rnn import pack_padded_sequence
import pickle

In [2]:
data_path = 'd:\\IG Data\\'
root = os.getcwd()
file_dict = create_file_paths_dict(data_path)
#print(file_dict)
data_df = create_data_csv(file_dict)
os.chdir(root)

File Paths Saved in Dictionary
CSV File Created.!
Contains Four Coulumns:

Type: Type of the Profile || User: username || Image: Full path of image file || Caption: Full path of text file


In [3]:
transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

dataset = IG_img_caption_dataset(
    csv_file = data_df,
    type_prof = 'trial',
    vocab=vocab,
    transform = transformer)

dataloader = DataLoader(dataset=dataset,
batch_size=2,
num_workers=0,
collate_fn=collate_fn,
shuffle=True)

In [4]:
len(dataset)

634

In [5]:
# Build the models
#torch.cuda.clear_memory_allocated()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embed_size = 10
learning_rate = 0.1 
num_epochs = 1
hidden_size = 2
num_layers = 2
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, hidden_size, len(vocab), num_layers).to(device)
    
    # Loss and optimizer
criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.bn.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(params, lr=learning_rate)
    
    # Train the models
total_step = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, captions, lengths) in enumerate(dataloader):
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
            
            # Forward, backward and optimize
        features = encoder(images)
        outputs = decoder(features, captions, lengths)
        loss = criterion(outputs, targets)
        decoder.zero_grad()
        encoder.zero_grad()
        loss.backward()
        optimizer.step()

            # Print log info
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Perplexity: {:5.4f}'
                      .format(epoch, num_epochs, i, total_step, loss.item(), np.exp(loss.item()))) 

ModuleAttributeError: 'EncoderCNN' object has no attribute 'linear'

In [ ]:
model = Word2Vec(reviews,size=100, window=5, min_count=5, workers=4)
weights = torch.FloatTensor(model.wv.vectors)
embedding = nn.Embedding.from_pretrained(weights)